### Install Dependencies

In [73]:
%pip install nltk

### Imports

In [74]:
import nltk
nltk.download('punkt')
import os
import zipfile
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sumed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Download NLTK Packages

In [75]:
if not os.path.exists('./packages'):
    os.makedirs('./packages')
nltk.data.path.append('./packages')
nltk.download('all', download_dir='./packages')
with zipfile.ZipFile('./packages/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('./packages/corpora')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to ./packages...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to ./packages...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     ./packages...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     ./packages...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to ./packages...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to ./packages...
[nltk_data]    |   Package bcp47 is already up-to-date!
[nltk_data]    | Downloading package biocreative

## Import Data

In [76]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./../../data/augmented_intents.json').read()
intents = json.loads(data_file)

## Check Data

In [77]:
intents.keys()

dict_keys(['intents'])

In [78]:
intents['intents'][:5]

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??',
   '??? ??? ??',
   'Is anyone thither?',
   'Hullo',
   'How be you?',
   'how make up ya',
   'Ripe day',
   'how are ya',
   'Hi',
   'heyy',
   'How do you do',
   'Exist anyone there?',
   'Safe day',
   'Represent anyone there?',
   "What ' s upwards",
   'Well day',
   'Hello',
   "What ' s upward",
   'whatsup',
   'Aloha state',
   'How are you?'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg',
   'Have a Unspoilt 24 hour interval',
   'talk to you after',
   'see you',
   'ttyl',
   'image you',
   'ace start out to g

##  Data pre-processing

In [79]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        # add to classes
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Lemmatization

In [80]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)


1394 documents
38 classes ['admission', 'canteen', 'college intake', 'committee', 'computerhod', 'course', 'creator', 'document', 'event', 'extchod', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod', 'hostel', 'hours', 'infrastructure', 'ithod', 'library', 'location', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'sem', 'sports', 'swear', 'syllabus', 'task', 'uniform', 'vacation']
686 unique lemmatized words ["'", "'s", '(', ')', '-', '/', '1', '24', '3rd', '53', '58', '89', 'a', 'about', 'ac', 'accept', 'access', 'accession', 'account', 'ace', 'acquire', 'act', 'actinium', 'action', 'active', 'activity', 'address', 'adeptness', 'adieu', 'admision', 'admission', 'admittance', 'adroitness', 'affair', 'after', 'against', 'agency', 'agenda', 'ai', 'ai/ml', 'aid', 'alive', 'allocation', 'allotment', 'allow', 'aloha', 'aloofness', 'alternating', 'am', 'an', 'and', 'ane', 'antiragging', 'any', 'anyone', 'apart', 'applied', 'a

a pickle file is made that will be used for prediction.

In [81]:
pickle.dump(words,open('./bin/words.pkl','wb'))
pickle.dump(classes,open('./bin/classes.pkl','wb'))

# Create training and testing data
Now that the training data has been created, the input and output will be included. The pattern will serve as our input, and the class to which the pattern belongs will serve as our output. However, since the machine cannot interpret language, we must convert it to numbers.

In [82]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:,0]), dtype=np.float32)
train_y = np.array(list(training[:,1]), dtype=np.float32)
print("Training data created")

Training data created


# Build the model

In [83]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
#decay was depreciated thus changes made according to latest version

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Learning rate schedule
initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Using Adam optimizer with learning rate schedule
adam = Adam(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
279/279 [==============================] - 1s 1ms/step - loss: 2.8538 - accuracy: 0.2590
Epoch 2/200
279/279 [==============================] - 0s 1ms/step - loss: 1.9373 - accuracy: 0.4907
Epoch 3/200
279/279 [==============================] - 0s 1ms/step - loss: 1.6840 - accuracy: 0.5502
Epoch 4/200
279/279 [==============================] - 0s 1ms/step - loss: 1.5004 - accuracy: 0.6184
Epoch 5/200
279/279 [==============================] - 0s 1ms/step - loss: 1.4030 - accuracy: 0.6334
Epoch 6/200
279/279 [==============================] - 0s 1ms/step - loss: 1.3198 - accuracy: 0.6492
Epoch 7/200
279/279 [==============================] - 0s 1ms/step - loss: 1.2002 - accuracy: 0.6987
Epoch 8/200
279/279 [==============================] - 0s 1ms/step - loss: 1.1786 - accuracy: 0.6923
Epoch 9/200
279/279 [==============================] - 0s 1ms/step - loss: 1.1880 - accuracy: 0.6980
Epoch 10/200
279/279 [==============================] - 0s 1ms/step - loss: 1.1998 - accura

## Compile & Save

In [87]:
model.save('./bin/model.h5', hist)
print("model created")

model created


## Test

In [88]:
model = load_model('./bin/model.h5')
intents = json.loads(open('./../../data/augmented_intents.json').read())
words = pickle.load(open('./bin/words.pkl','rb'))
classes = pickle.load(open('./bin/classes.pkl','rb'))

def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag(sentence, words, show_details=True):
    sentence_words = clean_sentence(sentence)

    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def classify(sentence, model):
    p = bag(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]

    ERROR_THRESHOLD = 0.1

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']

    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = classify(text, model)
    res = getResponse(ints, intents)
    return res

In [94]:
chatbot_response('homla')

1/1 [==============================] - 0s 18ms/step


'Good to see you again!'